Intini Alessia s309895 `<s309895@studenti.polito.it>`\
[`https://github.com/AlessiaIntini/Computational_Intelligence.git`](https://github.com/AlessiaIntini/Computational_Intelligence.git)  
Development lab #1 with Antonio Iorio s317748

In [66]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil
import numpy as np

We have decided to use a fixed sets of value to compare the number of steps to resolve the set covering across the 3 different h function. Because when we use a random values we can't compare the efficiency of the every h functions.\
The fixed sets:
```python
PROBLEM_SIZE = 10  #numero di elementi in ogni array
NUM_SETS = 20  #numero di array
SETS = (np.array([False,  True, False,  True, False, False, False, False, False,
       False]), np.array([ True,  True, False, False, False, False, False,  True, False,
        True]), np.array([ True, False,  True, False,  True, False, False, False,  True,
       False]), np.array([False,  True, False, False, False, False, False,  True, False,
        True]), np.array([ True,  True,  True,  True, False, False, False,  True, False,
       False]), np.array([False, False, False,  True, False, False, False, False, False,
       False]), np.array([False, False, False, False, False, False,  True, False, False,
       False]), np.array([ True, False,  True, False, False, False, False, False, False,
       False]), np.array([False,  True,  True,  True,  True, False, False, False,  True,
       False]), np.array([False, False, False, False, False, False, False, False, False,
       False]), np.array([False, False,  True, False, False, False,  True, False, False,
        True]), np.array([False,  True,  True, False, False,  True, False, False,  True,
       False]), np.array([False, False,  True, False, False, False, False, False,  True,
        True]), np.array([False, False,  True, False,  True, False, False, False, False,
       False]), np.array([ True,  True,  True,  True, False, False,  True, False, False,
        True]), np.array([False, False, False, False, False, False,  True, False, False,
        True]), np.array([ True,  True,  True, False, False,  True, False, False, False,
       False]), np.array([False,  True,  True, False, False, False, False, False,  True,
       False]), np.array([ True,  True, False,  True, False, False,  True, False, False,
       False]), np.array([False, False, False,  True, False, False,  True,  True, False,
        True]))
```

In [67]:
PROBLEM_SIZE = 100  #numero di elementi in ogni array
NUM_SETS = 200  #numero di array
# SETS = (np.array([False,  True, False,  True, False, False, False, False, False,
#        False]), np.array([ True,  True, False, False, False, False, False,  True, False,
#         True]), np.array([ True, False,  True, False,  True, False, False, False,  True,
#        False]), np.array([False,  True, False, False, False, False, False,  True, False,
#         True]), np.array([ True,  True,  True,  True, False, False, False,  True, False,
#        False]), np.array([False, False, False,  True, False, False, False, False, False,
#        False]), np.array([False, False, False, False, False, False,  True, False, False,
#        False]), np.array([ True, False,  True, False, False, False, False, False, False,
#        False]), np.array([False,  True,  True,  True,  True, False, False, False,  True,
#        False]), np.array([False, False, False, False, False, False, False, False, False,
#        False]), np.array([False, False,  True, False, False, False,  True, False, False,
#         True]), np.array([False,  True,  True, False, False,  True, False, False,  True,
#        False]), np.array([False, False,  True, False, False, False, False, False,  True,
#         True]), np.array([False, False,  True, False,  True, False, False, False, False,
#        False]), np.array([ True,  True,  True,  True, False, False,  True, False, False,
#         True]), np.array([False, False, False, False, False, False,  True, False, False,
#         True]), np.array([ True,  True,  True, False, False,  True, False, False, False,
#        False]), np.array([False,  True,  True, False, False, False, False, False,  True,
#        False]), np.array([ True,  True, False,  True, False, False,  True, False, False,
#        False]), np.array([False, False, False,  True, False, False,  True,  True, False,
#         True]))

SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

State = namedtuple('State', ['taken', 'not_taken'])


In [68]:

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [69]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [70]:

def h1(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def distance(state):
    return len(state.taken) + h3(state)

Result of execution are:
 - h1 resolve it in 85 steps
 - h2 resolve it in 9 steps
 - h3 resolve it in 9 steps

In [71]:
# frontier = PriorityQueue()
frontier = PriorityQueue()
#set() crea un set di oggetti
state = State(set(), set(range(NUM_SETS))) #State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
frontier.put((distance(state), state)) #(10, State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state): 
    counter += 1 #quante operazioni ci sono volute
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((distance(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

KeyboardInterrupt: 

In [ ]:
current_state

State(taken={2, 110, 54}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199})

In [ ]:
SETS[0]

array([False, False, False, False, False, False, False, False, False,
       False])